In [1]:
# importing what is important
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForMaskedLM
import emoji
import unicodedata
import re
import numpy as np
import pandas as pd
import string
from nltk.corpus import stopwords
from transformers import BertModel
##nltk.download('stopwords')


C:\Users\MrSwi\anaconda3\envs\projects\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class BertModel(nn.Module):
    """
    BERT :) 
    """
    
    def __init__(self, freeze=False):
        """
        
        """
        super(BertModel, self).__init__()
        D_in = 32000 # bert_in ?
        H, D_out= 16000,2
        
        self.bert =  AutoModelForMaskedLM.from_pretrained("asafaya/bert-mini-arabic")
        
        self.classifier = nn.Sequential(
            nn.Linear(D_in, H),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(H, D_out)
        )
        if freeze:
            for param in self.bert.parameters():
                param.requires_grad = False
                
    def forward(self, input_ids, attention_mask):
        
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask)
        
        last_hidden_state_cls = outputs[0][:, 0, :]
        
        logits = self.classifier(last_hidden_state_cls)
        
        return logits

In [10]:
def load_model(model_path):
    """
    Loading bert model
    @param model_path(str): the bert model path
    @return bert_model : a trained bert_model from model_path
    @return tokenizer : for preproccing_bert
    """
    # Importing model pretrained
    tokenizer = AutoTokenizer.from_pretrained("asafaya/bert-mini-arabic")
    bert_model = BertModel(freeze=False)
    bert_model.load_state_dict(torch.load("bert.pt"))
    return bert_model,tokenizer

In [11]:
bert_model,tokenizer = load_model("bert.pt")

In [5]:
punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ''' + string.punctuation

# Arabic stop words with nltk
stop_words = stopwords.words()

arabic_diacritics = re.compile("""
                             ّ    | # Shadda
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)

def text_preprocessing(text):
    '''
    text is an arabic string input
    
    the preprocessed text is returned
    '''
    
    #remove punctuations
    translator = str.maketrans('', '', punctuations)
    text = text.translate(translator)
    
    # remove Tashkeel
    text = re.sub(arabic_diacritics, '', text)
    
    #remove longation
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)

    text = ' '.join(word for word in text.split() if word not in stop_words)

    return text

In [6]:
def preprocessing_for_bert(data, tokinizer, text_preprocessing_fn = text_preprocessing, MAX_LEN=10):
    """Perform required preprocessing steps for pretrained BERT.
    @param    data (np.array): Array of texts to be processed.
    @return   input_ids (torch.Tensor): Tensor of token ids to be fed to a model.
    @return   attention_masks (torch.Tensor): Tensor of indices specifying which
                  tokens should be attended to by the model.
    """
    # Create empty lists to store outputs
    input_ids = []
    attention_masks = []
    
    # For every sentence...
    for i,sent in enumerate(data):
        # `encode_plus` will:
        #    (1) Tokenize the sentence
        #    (2) Add the `[CLS]` and `[SEP]` token to the start and end
        #    (3) Truncate/Pad sentence to max length
        #    (4) Map tokens to their IDs
        #    (5) Create attention mask
        #    (6) Return a dictionary of outputs
        encoded_sent = tokenizer.encode_plus(
            text=text_preprocessing_fn(sent),  # Preprocess sentence
            add_special_tokens=True,        # Add `[CLS]` and `[SEP]`
            max_length=MAX_LEN,                  # Max length to truncate/pad
            padding='max_length',        # Pad sentence to max length
            #return_tensors='pt',           # Return PyTorch tensor
            return_attention_mask=True,     # Return attention mask
            truncation = True 
            )
        # Add the outputs to the lists
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))
    # Convert lists to tensors
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)

    return input_ids, attention_masks

In [7]:
def get_prediction(model,text):
    """
    get prediction from model by text
    @param model(BertModel):
    @param text(str): a text of something to predict
    @return pred(int): the catogery of prediction by default (0: negative, 1:positive)
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    # Tracking variables
    text = text_preprocessing(text)
    text = np.array([text])
    b_input_ids, b_attn_mask = preprocessing_for_bert(text, tokenizer)
    with torch.no_grad():
        logits = model(b_input_ids, b_attn_mask)

        # Compute loss
        # Get the predictions
        pred = torch.argmax(logits, dim=1).flatten()
    return preds.item()
            

In [17]:
def display_prediction(text):
    
    pred = get_prediction(bert_model,text)
    print("Your text: " , text)
    if(pred==0):
        print("that is a NEGATIVE talk")
    else:
        print("that is a POSITIVE talk")

In [22]:
display_prediction(" ممكن تقولي يادكتور ، انت ومنهم على شاكلتك ، ماذا فعلتم لاوطانكم ؟ ولَك ياملعون شو اخترعت ...")

Your text:   ممكن تقولي يادكتور ، انت ومنهم على شاكلتك ، ماذا فعلتم لاوطانكم ؟ ولَك ياملعون شو اخترعت ...
that is a NEGATIVE talk


In [23]:
display_prediction("ستظل مسألة حقوق المرأة مسألة شائكة خاصة بدول الشرق اوسطية التي اغلبها لم تنفك من التص...")

Your text:  ستظل مسألة حقوق المرأة مسألة شائكة خاصة بدول الشرق اوسطية التي اغلبها لم تنفك من التص...
that is a NEGATIVE talk


In [31]:
#FALSE
display_prediction("أحد أسباب نجاح الإمارات أن كل شخص في هذه الدولة يعشق ترابها. نحن نحب الإمارات. ومضات من فكر. نصائح ل...")

Your text:  أحد أسباب نجاح الإمارات أن كل شخص في هذه الدولة يعشق ترابها. نحن نحب الإمارات. ومضات من فكر. نصائح ل...
that is a NEGATIVE talk


In [29]:
display_prediction("السعودية الشرقية القطيف . أعجبني الموقع المتميز بالهدوء وتوفر جميع الخدمات ياس مولعالم فراريووتر وورلد...")

Your text:  السعودية الشرقية القطيف . أعجبني الموقع المتميز بالهدوء وتوفر جميع الخدمات ياس مولعالم فراريووتر وورلد...
that is a POSITIVE talk


In [30]:
display_prediction("جيد جدا. أعجبني مطعم السلام نظافته وطهيه لكن أتمنى أن لايقتصر على البوفيه المفتوح بل نريد منه توصيل ...")

Your text:  جيد جدا. أعجبني مطعم السلام نظافته وطهيه لكن أتمنى أن لايقتصر على البوفيه المفتوح بل نريد منه توصيل ...
that is a POSITIVE talk
